In [3]:



import nltk
#nltk.download('popular')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from flask import Flask, render_template, request,send_file
from PIL import Image
import os
from keras.models import load_model
model = load_model('model.pkl')
import json
import random
import tensorflow
from tensorflow.keras import models
import cv2
intents = json.loads(open('data2.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))
IMAGE_SIZE = 256

model1 = models.load_model('model1')
model2 = models.load_model('model2')
model3=models.load_model('model3')

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def generate_form(fields):
    form_html = '<form method="POST" action="/get">'
    for field in fields:
        form_html += f'<label for="{field["name"]}">{field["label"]}</label><br>'
        if field["type"] == "radio":
            for option in field["options"]:
                form_html += f'<input type="radio" name="{field["name"]}" value="{option["value"]}">{option["label"]}<br>'
            
    form_html += '<input type="submit" value="Submit"></form>'
    return form_html



def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            if 'form' in i:
                form = i['form']
                form_html = generate_form(form['fields'])
                return form_html
            else:
                result = random.choice(i['responses'])
                break
    return result

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# def getResponse(ints, intents_json):
#     tag = ints[0]['intent']
#     list_of_intents = intents_json['intents']
#     for i in list_of_intents:
#         if(i['tag']== tag):
#             result = random.choice(i['responses'])
#             break
#     return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res
final_prediction = []

def disease_prediction(image):
  final_prediction.clear()  
  img_bytes = image.read()  # Read the image bytes
  nparr = np.frombuffer(img_bytes, np.uint8)  # Convert bytes to NumPy array
  img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)  
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE)) 
  predictions_model1 = model1.predict(np.array([img]))
  predicted_class_model1 = np.argmax(predictions_model1)
  
  predictions_model2 = model2.predict(np.array([img]))
  predicted_class_model2 = np.argmax(predictions_model2)
  
  predictions_model3 = model3.predict(np.array([img]))
  predicted_class_model3 = np.argmax(predictions_model3)
  
  if predicted_class_model1 == 0:
    final_prediction.append("cataract")
  elif predicted_class_model1 == 1:
    final_prediction.append("normal")

  if predicted_class_model2 == 0:
    final_prediction.append("Jaundice")
  elif predicted_class_model2 == 1:
    final_prediction.append("normal")
    
  if predicted_class_model3 == 0:
    final_prediction.append("Strabismus")
  elif predicted_class_model3 == 1:
    final_prediction.append("normal")  
    


app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
     userText = request.args.get('msg') 
     if userText=="cataractCheck" or userText=="JaundiceCheck" or userText=="StrabismusCheck" or userText=="full":
         return final_prediction
     return chatbot_response(userText)



@app.route("/upload",methods=['POST'])
def get_bot_response2():
    if 'image' in request.files :
      image=request.files['image']
      
      #img=image.save(os.path.join(os.getcwd(), 'image.jpg'))
      disease_prediction(image)
      return "mk"
    else :
        return "lk"


if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Jul/2023 12:01:49] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 240ms/step


127.0.0.1 - - [23/Jul/2023 12:01:55] "GET /get?msg=yes HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2023 12:18:43] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 202ms/step


...

127.0.0.1 - - [23/Jul/2023 12:31:23] "GET /get?msg=check HTTP/1.1" 200 -


1/1 [==============================] - 0s 103ms/step


127.0.0.1 - - [23/Jul/2023 12:31:37] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2023 12:31:40] "GET /get?msg=full HTTP/1.1" 200 -


1/1 [==============================] - 0s 48ms/step


127.0.0.1 - - [23/Jul/2023 12:31:53] "GET /get?msg=help HTTP/1.1" 200 -
